# Public Transport Victoria (PTV) Timetable API v3
To obtain an ```id/key``` pair please visit the [PTV API website](https://www.ptv.vic.gov.au/footer/data-and-reporting/datasets/ptv-timetable-api).

With them, you can create the required  ```secrets.json``` as follows:
``` python
credentials = {
    "id": "your id here",
    "key": "your key here"}
with open('secrets.json', 'w') as json_file:
    json.dump(credentials, json_file, indent=4)
```

## Preliminaries: making a request and authentication
For details on available endpoints and parameters, refer to [API documentation](https://timetableapi.ptv.vic.gov.au/swagger/ui/index).

Let us first instantiate a ```PTVv3``` class:

In [1]:
from ptvv3 import PTVv3
import json

with open('secrets.json', 'r') as json_file:
    credentials = json.load(json_file)
    ptv = PTVv3(credentials['id'], credentials['key'], debug=True)

Now let's test the API exercising the ```/v3/route_types``` endpoint:

In [2]:
import functools

@functools.cache
def get_route_typenames():
    data = ptv('/v3/route_types')
    d = {rt["route_type_name"].lower(): rt["route_type"] for rt in data['route_types']}
    return d | {v: k for k, v in d.items()}

route_typenames = get_route_typenames()
print(route_typenames)
ptv.debug = False

https://timetableapi.ptv.vic.gov.au/v3/route_types?devid=3002669&signature=b99d700e84f1a5eb5cfe853e13188625e7f406d3
{'train': 0, 'tram': 1, 'bus': 2, 'vline': 3, 'night bus': 4, 0: 'train', 1: 'tram', 2: 'bus', 3: 'vline', 4: 'night bus'}


## Plumbing primitives
We provide here wrappers around 8 endpoints, for collecting information regarding
* departures (```/v3/departures/route_type/{route_type}/stop/{stop_id}/route/{route_id}```),
* directions (```/v3/directions/route/{route_id}``` and ```/v3/directions/{direction_id}/route_type/{route_type}```),
* routes (```/v3/routes/{route_id}```),
* runs (```/v3/runs/route/{route_id}/route_type/{route_type}``` and ```/v3/runs/{run_ref}/route_type/{route_type}```) and
* stops (```/v3/stops/route/{route_id}/route_type/{route_type}``` and ```/v3/stops/location/{latitude},{longitude}```).

The full API documentation can be found [here](https://timetableapi.ptv.vic.gov.au/swagger/ui/index).

From a high-level:
* a *route* is an ordered collection of *stops* that can run in more than one *direction*.
* A *run* represents a vehicle travelling along a *route* in a *direction*.
* **TODO** Add/understand if departures is predictive or not.

With the above primitives, a possible workflow is:
* from location (latitude, longitude) to *stops*
* from *stop* to *routes*
* from *route* to *directions*
* from *route* to *runs*
* from *route*/*stop* to *departures*

In the following, we'll use the following identifiers to test endpoints:

In [3]:
route_type = 2  # bus (routes, stops, runs)
stop_id = 22833  # Willgilson Ct/Golf Links Ave, Oakleigh
route_id = 13271  # bus 733 service from Oakleigh to Box Hill
direction_id = 30  # from Oakleigh towards Box Hill (opposite is 182)

### Departures
* Endpoint: ```/v3/departures/route_type/{route_type}/stop/{stop_id}/route/{route_id}``` <br /> *Service departures from the specified stop for the specified route (and route type); departures are timetabled and real-time (if applicable).*

In [ ]:
"""
Optional parameters:
* direction_id id
* gtfs bool
* date_utc iso8601utc
* max_results int
* include_cancelled bool
* look_backwards bool
* expand string
* include_geopath bool
"""

departures = ptv(f'/v3/departures/route_type/{route_type}/stop/{stop_id}/route/{route_id}', expand='All')  #, include_geopath='true')

In [ ]:
departures.keys()

### Directions
* ```/v3/directions/route/{route_id}```
* ```/v3/directions/{direction_id}/route_type/{route_type}```

No parameters

In [ ]:
directions_route = ptv(f'/v3/directions/route/{route_id}')

In [ ]:
directions_route

In [ ]:
directions = ptv(f'/v3/directions/{direction_id}/route_type/{route_type}')

In [ ]:
directions

### Routes
* ```/v3/routes/{route_id}```

In [ ]:
routes = ptv(f'/v3/routes/{route_id}', include_geopath='true')

In [ ]:
routes

### Runs
* ```/v3/runs/route/{route_id}/route_type/{route_type}```
* ```/v3/runs/{run_ref}/route_type/{route_type}```

In [ ]:
runs_route = ptv(f'/v3/runs/route/{route_id}/route_type/{route_type}', expand='All', include_geopath='true')

In [ ]:
for index, run in enumerate(runs_route['runs']):
    if run['vehicle_position'] is not None:
        print(run)

In [ ]:
runs = ptv(f'/v3/runs/20-733--1-MF2-215/route_type/{route_type}')

In [ ]:
runs

In [ ]:
ptv(f'/v3/runs/20-733--1-MF2-215', expand='All', include_geopath='true')

### Stops
* ```/v3/stops/route/{route_id}/route_type/{route_type}```
* ```/v3/stops/location/{latitude},{longitude}```

In [ ]:
def v3_stops_route():
    pass
    
def v3_stops_location():
    pass

In [ ]:
    if not isinstance(route_type, int):
        route_type = route_types_dict()[route_type.lower()]

### Mobility options at \<coordinates\>
Exercising the ```/v3/stops/location/{latitude},{longitude}``` endpoint to discover all unique travel options around a specific coordinate.

In [26]:
def stops_from_location(latitude, longitude, route_types=[], max_results=50, max_distance=200):
    request = f'/v3/stops/location/{latitude},{longitude}?'
    route_types = [route_types] if isinstance(route_types, int) else route_types
    parameters = [f'route_types={route_type}' for route_type in route_types]
    parameters.append(f'max_results={max_results}')
    parameters.append(f'max_distance={max_distance}')
    return do_request(request + '&'.join(parameters))


def print_unique_routes(stops):
    route_ids = []
    for stop in stops['stops']:

        # import pprint
        # pprint.pprint(stop)
        # break
        
        stop_typename = route_typenames[stop["route_type"]].title()
        stop_desc = f'{stop_typename} stop \x1b[32m{stop["stop_id"]}\x1b[0m {stop["stop_name"].strip()}, {stop["stop_suburb"]}, \x1b[34m{round(stop["stop_distance"], 2)}m\x1b[0m'
        stop_printed = False  # delay description for stops whose routes have already been printed
        
        for route in stop["routes"]:
            if route["route_id"] in route_ids:
                continue

            route_ids.append(route["route_id"])  # debug
            if not stop_printed:
                print(stop_desc)
                stop_printed = True

            service_number = ':' if not route["route_number"] else f' \x1b[31m{route["route_number"]}\x1b[0m:'
            print(f'  - {stop_typename}{service_number} {route["route_name"]} (id: {route["route_id"]})')

            # if route["route_number"] == '733':  # debug
            #     print(route)  # {... 'route_id': 13271, 'route_name': 'Oakleigh - Box Hill via Clayton & Monash University & Mt Waverley', 'route_number': '733', 'route_gtfs_id': '4-733'}

In [27]:
willgilson_ct = {'lat': -37.9057, 'lon': 145.0927}
monash_bus_hub = {'lat': -37.9143, 'lon': 145.1315}

stops = ptv(f'/v3/stops/location/{willgilson_ct["lat"]},{willgilson_ct["lon"]}', max_distance=560)
print_unique_routes(stops)

# stops = stops_from_location(willgilson_ct['lat'], willgilson_ct['lon'], route_types=0, max_distance=1000)
# print_unique_routes(stops)

Bus stop 22833 Willgilson Ct/Golf Links Ave, Oakleigh, 135.15m
  - Bus 733: Oakleigh - Box Hill via Clayton & Monash University & Mt Waverley (id: 13271)
Bus stop 16339 Westminster St/Burlington St, Oakleigh, 439.33m
  - Bus 900: Stud Park SC (Rowville) - Caulfield via Monash University & Chadstone (SMARTBUS Service) (id: 12753)
  - Bus 704: Oakleigh Station - Westall Station via Clayton (id: 13665)
  - Bus 742: Ringwood - Chadstone SC via Vermont South & Glen Waverley & Oakleigh (id: 14930)
Bus stop 22875 Best St/North Rd, Oakleigh, 511.25m
  - Bus 630: Elwood - Monash University via Gardenvale & Ormond & Huntingdale (id: 13067)
Bus stop 10005 Oakleigh SC/Hanover St, Oakleigh, 552.75m
  - Bus 862: Dandenong - Chadstone via North Dandenong & Oakleigh (id: 8922)
  - Bus 802: Dandenong - Chadstone via Mulgrave & Oakleigh (id: 8924)
  - Bus 804: Dandenong - Chadstone via Wheelers Hill & Oakleigh (id: 8934)
  - Bus 800: Dandenong - Chadstone via Princes Highway & Oakleigh (id: 13820)


### Active buses on a specified route

In [31]:
def print_active_buses(route):

    runs = ptv(f'/v3/runs/route/{route}/route_type/2', expand='All')['runs']
    
    for index, run in enumerate(runs):
        if run['vehicle_position'] is None:
            continue
    
        pos = run['vehicle_position']
        bearing = round(pos["bearing"]) if pos["bearing"] is not None else 'n/a'
        print(f'{index:>4}/{len(runs)}) \x1b[31mseq\x1b[0m {run["run_sequence"]:3} \x1b[31mref\x1b[0m {run["run_ref"]:17} \x1b[31mpos\x1b[0m {round(pos["latitude"], 4):.4f},{round(pos["longitude"], 4):.4f} \x1b[31mdir\x1b[0m {bearing:3} \x1b[31mdest\x1b[0m {run["destination_name"]}')


print_active_buses(13271)

In [38]:
runs = ptv('/v3/runs/route/13271', expand='All', date_utc='2024-03-20T03:46:22Z')

In [39]:
from collections import defaultdict

merged = defaultdict(list)
for run in runs['runs']:
    for k, v in run.items():
        if v not in merged[k]:
            merged[k].append(v)

merged

defaultdict(list,
            {'run_id': [-1],
             'run_ref': ['20-733--1-MF2-215',
              '20-733--1-MF2-51',
              '20-733--1-MF2-53',
              '20-733--1-MF2-217',
              '20-733--1-MF2-281',
              '20-733--1-MF2-115',
              '20-733--1-MF2-219',
              '20-733--1-MF2-55',
              '20-733--1-MF2-283',
              '20-733--1-MF2-117',
              '20-733--1-MF2-221',
              '20-733--1-MF2-57',
              '20-733--1-MF2-285',
              '20-733--1-MF2-119',
              '20-733--1-MF2-223',
              '20-733--1-MF2-287',
              '20-733--1-MF2-59',
              '20-733--1-MF2-225',
              '20-733--1-MF2-121',
              '20-733--1-MF2-289',
              '20-733--1-MF2-61',
              '20-733--1-MF2-333',
              '20-733--1-MF2-227',
              '20-733--1-MF2-123',
              '20-733--1-MF2-291',
              '20-733--1-MF2-63',
              '20-733--1-MF2-125',
    

In [ ]:
import numpy as np
from matplotlib import pyplot as plot

fig, ax = plot.subplots(figsize=(8, 8))
plot.title('Bus 733 -- Oakleigh Station/Johnson St')

paths = do_request('/v3/runs/20-733--1-MF2-95?expand=All&include_geopath=true')['runs'][0]['geopath'][0]['paths']
for index, path in enumerate(paths):
    str_coords = path.replace(', ', ',').split()
    coords = np.array([list(map(float, str_coord.split(','))) for str_coord in str_coords])
    ax.plot(coords[:, 1], coords[:, 0], ['k-', 'r-'][index])
    # ax.plot(coords[:, 1], coords[:, 0], ['k.', 'r.'][index])

ax.annotate('Oakleigh', xy=(145.0873714, -37.9005191), xycoords='data', xytext=(-2, 2), textcoords='offset points', ha='right')
ax.annotate('Monash', xy=(145.1315377, -37.9142599), xycoords='data', xytext=(2, 2), textcoords='offset points')
ax.axis('equal')
plot.show()

### Links
* [Timetable API website](https://www.ptv.vic.gov.au/footer/data-and-reporting/datasets/ptv-timetable-api/)
* [Timetable API documentation](https://timetableapi.ptv.vic.gov.au/swagger/ui/index)
* [Smartrak OBD II brochure](https://go.smartrak.com/rs/040-SMS-890/images/PDF-Product-Brochure-1199-OBD-II.pdf)

### Implementations
* [pyptv3](https://pypi.org/project/pyptv3/) (python)
* [PtvTimetableApi](https://github.com/njmh/ptv-timetable-api/blob/master/src/PtvTimetableApi.php) (PHP)

In [ ]:
monash_bus_station = {'lat': -37.9142599, 'lon': 145.1315377}

stops = stops_from_location(monash_bus_station['lat'], monash_bus_station['lon'], max_distance=100)
print_unique_routes(stops)